---
**Sentiment Analysis on Social Media(Twitter)**

---

In [10]:
# Install Kaggle library
!pip install kaggle


# Set up Kaggle API credentials

In [11]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
# Download the Sentiment140 dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


# Extract the downloaded dataset

In [13]:
from zipfile import ZipFile
df = '/content/sentiment140.zip'
with ZipFile(df, 'r') as zip:
  zip.extractall()
  print('Dataset is Extracted')

Dataset is Extracted


----
# Importing necessary libraries
---

In [15]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# importing the dataset

In [17]:
twitter_dataset =  pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'latin-1')

In [18]:
twitter_dataset.shape

(1599999, 6)

In [19]:
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


---
# Data preprocessing
---

In [20]:
#  Assigning column names to the dataset
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding = 'latin-1')

In [21]:
twitter_dataset.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [23]:
# Checking for missing values
twitter_dataset.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [24]:
# Checking the number of labels values
twitter_dataset['target'].value_counts()

,count
target,
0,800000
4,800000


In [26]:
#Replace sentiment labels (4 to 1 for positive sentiment)
twitter_dataset.replace({'target':{4:1}}, inplace = True)

In [27]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
1,800000


0-> Negative Tweet

1-> Positive Tweet

# Applying porter stemmer

In [28]:
ps = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [29]:
# Apply the stemming function to the tweet text
twitter_dataset['stemmed_content']  = twitter_dataset['text'].apply(stemming)

In [30]:
twitter_dataset.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


# Spliting dataset into features and labels

In [31]:
X = twitter_dataset['stemmed_content'].values
y = twitter_dataset['target'].values


print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [32]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size =0.2, stratify = y, random_state = 42)


# Converting the textual data to numerical data by Tf-idfVectorizer

In [33]:
VCT = TfidfVectorizer()
X_train = VCT.fit_transform(X_train)
X_test = VCT.transform(X_test)

In [34]:
print(X_train)

  (0, 307108)	0.46206048815324474
  (0, 239679)	0.15130037108228483
  (0, 146067)	0.12929728405657018
  (0, 175252)	0.224070805470346
  (0, 128605)	0.22108856600702773
  (0, 4832)	0.317074267861159
  (0, 124524)	0.18318401951949756
  (0, 205794)	0.24140229063801746
  (0, 454381)	0.20169626473577715
  (0, 286478)	0.16123218610004272
  (0, 406297)	0.2978221095272138
  (0, 220296)	0.43015677907624866
  (0, 388138)	0.20555120011808467
  (0, 154767)	0.26976607043258233
  (1, 445870)	0.6361096685891185
  (1, 161801)	0.5778049407933611
  (1, 124611)	0.5113765148324884
  (2, 125319)	0.6383069130836649
  (2, 349409)	0.22232944888223494
  (2, 444761)	0.30331529032956345
  (2, 358186)	0.19837942712286838
  (2, 267649)	0.19309660201644555
  (2, 12436)	0.2529872032123258
  (2, 453420)	0.2347069337186747
  (2, 312657)	0.3154702974657607
  :	:
  (1279997, 124611)	0.2537781914144255
  (1279997, 301683)	0.1908678391932645
  (1279997, 168646)	0.20490659397970187
  (1279997, 156392)	0.23570311036994007
 

In [35]:
print(X_test)

  (0, 28874)	0.1778395103911245
  (0, 43712)	0.23562815302828183
  (0, 78636)	0.5158100011206617
  (0, 96399)	0.255967788489452
  (0, 97585)	0.4019235611854435
  (0, 129417)	0.25650960779862714
  (0, 189057)	0.31324918577405797
  (0, 240451)	0.15341308097014625
  (0, 334643)	0.14719329779308424
  (0, 335577)	0.22602158147814247
  (0, 384697)	0.3281164007446601
  (0, 435956)	0.14183025329879742
  (0, 453357)	0.1781708363247895
  (1, 35118)	0.3128685670821343
  (1, 36669)	0.2729958846742327
  (1, 39445)	0.16867093960211466
  (1, 57115)	0.19043301054662504
  (1, 74274)	0.21148120876702692
  (1, 78790)	0.13386322067407883
  (1, 170374)	0.17525273735418329
  (1, 171245)	0.12468774856570086
  (1, 183279)	0.24586158827112847
  (1, 233854)	0.3852709938491561
  (1, 240223)	0.1674195650536303
  (1, 301683)	0.13212235134015302
  :	:
  (319997, 135536)	0.21809964977532384
  (319997, 146067)	0.31485892246365776
  (319997, 169469)	0.1945044482645627
  (319997, 225096)	0.2933179603743362
  (319997, 3

# Training the model

In [36]:
classifier = LogisticRegression(max_iter =  1000)
classifier.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

# Model Evaluation

# Evaluate the model on training data

In [37]:
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [38]:
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.7999984375


# Evaluate the model on test data

In [39]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [42]:
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.776796875


In [43]:
# Making confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, X_test_prediction)
print(cm)
accuracy_score(y_test, X_test_prediction)

[[121053  38947]
 [ 32478 127522]]


0.776796875

# saving the model

In [44]:
import pickle
filename = 'sentiment_analysis_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

# using a saved model for future model

In [46]:
loaded_model =  pickle.load(open('/content/sentiment_analysis_model.sav', 'rb'))


---
# Testing the model on a new example
---

In [50]:
X_new = X_test[200]
print(y_test[200])
prediction =  loaded_model.predict(X_new)
print(prediction)

if (prediction[0] ==  0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet


In [53]:
X_new = X_test[3]
print(y_test[3])
prediction =  loaded_model.predict(X_new)
print(prediction)

if (prediction[0] ==  0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet
